
⚡ Building language agents as graphs ⚡

## Overview

[LangGraph](https://langchain-ai.github.io/langgraph/) is a library for building stateful, multi-actor applications with LLMs.
Inspired by [Pregel](https://research.google/pubs/pub37252/) and [Apache Beam](https://beam.apache.org/), LangGraph lets you coordinate and checkpoint multiple chains (or actors) across cyclic computational steps using regular python functions (or [JS](https://github.com/langchain-ai/langgraphjs)). The public interface draws inspiration from [NetworkX](https://networkx.org/documentation/latest/).

The main use is for adding **cycles** and **persistence** to your LLM application. If you only need quick Directed Acyclic Graphs (DAGs), you can already accomplish this using [LangChain Expression Language](https://python.langchain.com/docs/expression_language/).

Cycles are important for agentic behaviors, where you call an LLM in a loop, asking it what action to take next.

## Installation

```shell
pip install -U langgraph
```

## Quick start

One of the central concepts of LangGraph is state. Each graph execution creates a state that is passed between nodes in the graph as they execute, and each node updates this internal state with its return value after it executes. The way that the graph updates its internal state is defined by either the type of graph chosen or a custom function.

State in LangGraph can be pretty general, but to keep things simpler to start, we'll show off an example where the graph's state is limited to a list of chat messages using the built-in `MessageGraph` class. This is convenient when using LangGraph with LangChain chat models because we can directly return chat model output.

First, install the LangChain OpenAI integration package:

```python
pip install langchain_openai
```

We also need to export some environment variables:

```shell
export OPENAI_API_KEY=sk-...
```

And now we're ready! The graph below contains a single node called `"oracle"` that executes a chat model, then returns the result:

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()
groq_api_key: str = os.environ["GROQ_API_KEY"]

In [2]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

model = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

graph = MessageGraph()

graph.add_node("oracle", model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

In [3]:
runnable.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', id='6ac201ea-bb79-4f0b-bf20-b697bee4140e'),
 AIMessage(content="The sum of 1 + 1 is 2.\n\nHere's the step-by-step addition:\n\n1. Write down the numbers to be added, aligning them by place value:\n\n1\n1\n\n2. Add the columns from right to left:\n\n1 + 1 = 2\n\nSo, 1 + 1 = 2.", response_metadata={'token_usage': {'completion_time': 0.145, 'completion_tokens': 83, 'prompt_time': 0.004, 'prompt_tokens': 18, 'queue_time': None, 'total_time': 0.149, 'total_tokens': 101}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-281eea61-506c-433d-8b18-3ef45e0f8685-0')]


So what did we do here? Let's break it down step by step:

1. First, we initialize our model and a `MessageGraph`.
2. Next, we add a single node to the graph, called `"oracle"`, which simply calls the model with the given input.
3. We add an edge from this `"oracle"` node to the special string `END` (`"__end__"`). This means that execution will end after the current node.
4. We set `"oracle"` as the entrypoint to the graph.
5. We compile the graph, translating it to low-level [pregel operations](https://research.google/pubs/pregel-a-system-for-large-scale-graph-processing/) ensuring that it can be run.

Then, when we execute the graph:

1. LangGraph adds the input message to the internal state, then passes the state to the entrypoint node, `"oracle"`.
2. The `"oracle"` node executes, invoking the chat model.
3. The chat model returns an `AIMessage`. LangGraph adds this to the state.
4. Execution progresses to the special `END` value and outputs the final state.

And as a result, we get a list of two chat messages as output.

### Interaction with LCEL

As an aside for those already familiar with LangChain - `add_node` actually takes any function or [runnable](https://python.langchain.com/docs/expression_language/interface/) as input. In the above example, the model is used "as-is", but we could also have passed in a function:


In [4]:
# This will not work with MessageGraph!
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant named {name} who always speaks in pirate dialect"),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | model

# State is a list of messages, but our chain expects a dict input:
#
# { "name": some_string, "messages": [] }
#
# Therefore, the graph will throw an exception when it executes here.
graph.add_node("oracle", chain)

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.


ValueError: Node `oracle` already present.

In [5]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

model = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
model_with_tools = model.bind_tools([multiply])

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([multiply])
builder.add_node("multiply", tool_node)

builder.add_edge("multiply", END)

builder.set_entry_point("oracle")

In [6]:
from typing import Literal, List
from langchain_core.messages import BaseMessage

def router(state: List[BaseMessage]) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

builder.add_conditional_edges("oracle", router)

In [7]:
runnable = builder.compile()

runnable.invoke(HumanMessage("What is 123 * 456?"))

[HumanMessage(content='What is 123 * 456?', id='d230b18b-2127-485d-a64f-0178eda3ab28'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ht53', 'function': {'arguments': '{"first_number":123,"second_number":456}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.217, 'completion_tokens': 123, 'prompt_time': 0.256, 'prompt_tokens': 1135, 'queue_time': None, 'total_time': 0.473, 'total_tokens': 1258}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6fe5edc3-7a75-4e4e-8e8a-82fdf6802b7a-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_ht53'}]),
 ToolMessage(content='56088', name='multiply', id='65c5d4a6-10a5-49e2-b5c7-69cafb7a05b8', tool_call_id='call_ht53')]

In [8]:
runnable.invoke(HumanMessage("What is your name?"))

[HumanMessage(content='What is your name?', id='b161d674-1a19-4e9b-aef5-d47c4901967e'),
 AIMessage(content='I am a helpful assistant. How can I assist you today?\n\n(If you have a specific question or task that requires the use of a tool, please let me know and I will do my best to assist you using the appropriate tool.)', response_metadata={'token_usage': {'completion_time': 0.088, 'completion_tokens': 50, 'prompt_time': 0.223, 'prompt_tokens': 1103, 'queue_time': None, 'total_time': 0.311, 'total_tokens': 1153}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f7f202f-5826-406a-b391-7ad465af9612-0')]

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [10]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)

In [11]:

model = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

In [12]:
model = model.bind_tools(tools)

In [13]:
from typing import TypedDict, Annotated

def add_messages(left: list, right: list):
    """Add-don't-overwrite."""
    return left + right

class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages: Annotated[list, add_messages]

In [14]:
from typing import Literal

# Define the function that determines whether to continue or not
def should_continue(state: AgentState) -> Literal["action", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "action" node
    if last_message.tool_calls:
        return "action"
    # Otherwise, we stop (reply to the user)
    return "__end__"


# Define the function that calls the model
def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [15]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [16]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf'),
  AIMessage(content='The weather in San Francisco today is expected to be partly cloudy with a high of 64°F and a low of 53°F. There is a 10% chance of rain.', response_metadata={'token_usage': {'completion_time': 0.075, 'completion_tokens': 43, 'prompt_time': 0.488, 'prompt_tokens': 1093, 'queue_time': None, 'total_time': 0.563, 'total_tokens': 1136}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-c59550c6-dae8-44ae-bc48-1cb8f018c776-0')]}

In [17]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs, stream_mode="updates"):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='The weather in San Francisco today is expected to be partly cloudy with a high of 64°F and a low of 53°F. There is a 10% chance of rain.', response_metadata={'token_usage': {'completion_time': 0.076, 'completion_tokens': 43, 'prompt_time': 0.242, 'prompt_tokens': 1093, 'queue_time': None, 'total_time': 0.318, 'total_tokens': 1136}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-ecd4a8a8-5175-4934-8955-fb3e6dbb5872-0')]}

---



In [20]:
inputs = {"messages": [HumanMessage(content="what is the weather in Singapore today")]} 
async for output in app.astream_log(inputs, include_types=["llm"]):
    # astream_log() yields the requested logs (here LLMs) in JSONPatch format
    for op in output.ops:
        if op["path"] == "/streamed_output/-":
            # this is the output from .stream()
            ...
        elif op["path"].startswith("/logs/") and op["path"].endswith(
            "/streamed_output/-"
        ):
            # because we chose to only include LLMs, these are LLM tokens
            print(op["value"])

content='' additional_kwargs={'tool_calls': [{'id': 'call_7r5d', 'function': {'arguments': '{"query":"weather in Singapore today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'logprobs': None} id='run-a6bef77b-8a3c-4278-bcf3-f1f7a6f8c63b' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Singapore today'}, 'id': 'call_7r5d'}] tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '{"query":"weather in Singapore today"}', 'id': 'call_7r5d', 'index': None}]
content='The weather in Singapore today is partly cloudy with a temperature of 30.0 degrees Celsius (86 degrees Fahrenheit).' response_metadata={'finish_reason': 'stop', 'logprobs': None} id='run-e2bd909a-a64f-474b-ad03-2b0059d1408a'
